In [ ]:
from typing import List
import dask
import numpy as np
import dask.array as da
import dask.bag
from dask.distributed import Client

In [ ]:
import fsspec

In [ ]:
client = Client()
client

In [ ]:
def parallel_recursive_list(path: str, fs = None) -> List[str]:
    print(f"Listing {path}")
    if fs is None:
        fs = fsspec.core.url_to_fs(path)[0]
    info = []
    try:
        infos = fs.ls(path, detail=True)
    except PermissionError:
        return []
    files = []
    dirs = []
    for i in infos:
        if i["type"] == "directory":
            # You can speed this up by using futures, covered in "Advanced Scheduling"
            dir_list = dask.delayed(parallel_recursive_list)(i["name"], fs)
            dirs += dir_list
        else:
            files.append(i["name"])
    for sub_files in dask.compute(dirs):
        files.extend(sub_files)
    return files
    

In [ ]:
files = parallel_recursive_list("file:///tmp")

In [ ]:
files

In [ ]:
multiprocessing.cpu_count()